# Regresiones lineales

En este modulo aprenderemos a hacer regresiones lineales para cortes transversales. Un corte transversal puede pensarse como una fotografía (datos) en un momento dado. Se enseñara, entonces, a analizar estas fotografías. Las técnicas que vamos a aprender incluyen variables instrumentales, diferencias en diferencias, y regresiones discontinuas. Todas comparten el objetivo de reducir sesgos en los estimativos y acercarse un poco más (ojo no del todo) a relaciones causales entre las variables.

## Preliminares

Las regresiones pueden tener sesgos que evitan que podamos sobre-interpretar relaciones entre variables.

Primero importemos algunos paquetes

In [ ]:
from __future__ import print_function
# para estructura de datos
import pandas as pd
import numpy as np

# para gráficas e interacciones
import matplotlib 
from matplotlib import pyplot as plt
import seaborn as sn
import plotnine as p9
from plotnine import ggplot, geom_point, geom_line, aes, geom_smooth, facet_wrap, themes
import bqplot as bq
from bqplot import pyplot as bqplt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import GridspecLayout

# para estadística 
import statsmodels.formula.api as smf
import statsmodels.api as sm 
#from statsmodels.sandbox.regression.gmm import IV2SLS 
from linearmodels.iv import IV2SLS


Ahora cargemos algunas bases de datos

In [ ]:
# HISP: Health Insurance Subsidy Program
HISP = pd.read_stata('DataSets/HISP/evaluation.dta')
HISP_itr = pd.read_stata('DataSets/HISP/evaluation.dta', iterator=True) #tiene, entre otras cosas, la descripción de las variables
#print(HISP.dtypes) #columnas con tipo de datos 
#print('\n')
#var_temp = 'female_hh' #cambiar para obtener descripción de la columna
#print("La variable " + var_temp + " es: " + HISP_itr.variable_labels()[var_temp])

# Bangladesh data (Khandker et al 2010 worldbank book)
bang = pd.read_csv("DataSets/Khandker2010/hh_98.csv")
bang["lexptot"] = np.log(bang['exptot']) #log of expenditures
bang["lnland"] = np.log(1 + bang['hhland']/100) #log of land owned
#print(bang.dtypes) #columnas con tipo de datos 
#print('\n')


### Heterocedasticidad

Empecemos con un caso típico para entender por qué puede haber sesgos en las regresiones: heterocedasticidad. Pero primero veamos datos con homocedasticidad.

In [ ]:
n = 1000
X_data = np.linspace(1,100, n) 
Y_raw = 3.5 + 2.1 * X_data
Y_noise = np.random.normal(loc = 0, scale = 5, size = n) #loc es el promedio, scale es desviación estandar
Y = pd.DataFrame({"X": X_data, "Y": Y_raw + Y_noise})
model = smf.ols(formula='Y ~ X', data = Y)
results = model.fit()
predictions = results.predict(Y['X'])
residuals = results.resid

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10,5)) #One figure (fig) and two subplots (ax1, ax2)
ax1.plot(Y['X'], Y['Y'], 'ko')
ax1.plot(Y['X'], predictions, 'r', linewidth=3)
ax1.set(xlabel='X', ylabel='Y',
       title='Homoscedasticidad \n Ruido en medición parecido en todo x')
sm.qqplot(residuals, line = 'r', ax = ax2)
ax2.set(title='qqplot')
plt.show()
print(results.summary()) 
#Los coeficientes ¿Se parecen a los de Y_raw? 
#¡Si! Con homocedasticidad los estimativos lineales son bien cercanos


Ahora veamos con heterocedasticidad. Esto es, la desviación estandar de Y depende del nivel de X

In [ ]:
n = 500
X_data = np.linspace(1,100, n)
Y_raw = 1.37 + 2.097 * X_data
def f(k,robust = False): #Esta función tiene comandos para la regresión y para gráficas.
    f, axes = plt.subplots(1, 2, figsize=(12, 4))
    #El ruido puede ser una función complicada que depende del nivel de la variable independiente X
    Y_noise = np.random.normal(loc = 0, scale = X_data**k, size = n)
    Y = pd.DataFrame({"X": X_data, "Y": Y_raw + Y_noise})
    
    # Regresión
    model = smf.ols(formula='Y ~ X', data = Y)
    if robust == True:
        results = model.fit(cov_type='HC3') #errores standard robustos (a heterocedasticidad)
    else:
        results = model.fit() 
    residuals = results.resid
    Y['predictions'] = results.predict(Y['X'])

    #Gráficas
    sn.scatterplot(x = 'X', y='Y', data=Y, color = 'black', ax = axes[0])
    axes[0].set_title("Heterocedasticidad \n Ruido en medición depende del nivel de x")
    axes[0].plot(Y['X'], Y['predictions'], color = 'red')
    axes[1].axis('off')
    const = str(round(results.params[0],3))
    slope = str(round(results.params[1],3))
    axes[1].text(0, 0.5, 'Verdad: Intercepto: 1.370; Coef. X: 2.097 \nRegres: Intercepto: ' + const + '; Coef. X: ' + slope,
        color='black', fontsize=15)
    
    #Output
    plt.show()
    print(results.summary()) 
    

#Interactuar con gráficas
interact(f, k = widgets.FloatSlider(min=0, max=3, step=.2, value=1.25), 
         robust = widgets.Checkbox(value = False, description = 'Std. err. robustos'));

#Los coeficientes ¿Se parecen a los de Y_raw? 
#¡NO! Con heterocedasticidad, hay sesgo en los estimativos lineales, se pueden alejar de la realidad (incluso pueden ser significativos!)
# Hay correcciones. Por ejemplo, active y desactive la regresion con errores estandar robustos (a heterocedasticidad; ver model.fit en función f). 
# Cada vez que activa o desactiva, el ruido cambia (ver función f), 
# pero en general se reduce nuestra incertidumbre e.g. ver std. err intercepto


¿Cómo sabemos si nuestra data tiene heterocedasticidad? Cualitativamente con plots QQ pero también hay tests.

Usemos la data en HISP: Health Insurance Subsidy Program

In [ ]:
# Visualización
f, axes = plt.subplots(1, 2, figsize=(12, 4))
sn.scatterplot(x = 'poverty_index', y='health_expenditures', 
               size = 'educ_hh', #el tamaño de los puntos sigue el valor de esta columna
               data=HISP,
              ax = axes[0])
# La visualización parece mostrar con claridad la presencia de heterocedasticidad:
# Los más pobres en general gastan poco en salud pero a medida que mejora el 
# indice de pobreza hay mucha variabilidad.

# Regresión
# Vamos a hacer algo que no es necesario en este ejemplo, pero que vale la pena aprender.
# Dividimos la data en dos: train_data (para ajustar el modelo) y test_data (para predecir nuevos valores de y)
msk = np.random.rand(HISP.shape[0]) < 0.5 #Boolean index (mask)
train_data = HISP.loc[msk,:]
test_data = HISP.loc[~msk,:] #esto es para predecir (ver abajo)
if 'Intercept' not in test_data.columns:
    test_data.insert(0,'Intercept',1) 
if 'poverty_index:educ_hh' not in test_data.columns:
    test_data.insert(0,'poverty_index:educ_hh', test_data['poverty_index']*test_data['educ_hh'])
# Escribimos la formula con *, para incluir todas las interacciones y efectos principales
model = smf.ols(formula='health_expenditures ~ poverty_index*educ_hh', data = train_data) 
results = model.fit()
npred = 100 #cantidad de valores a predecir
predictions = results.predict(test_data.loc[:,['Intercept','poverty_index','educ_hh','poverty_index:educ_hh']])
axes[0].scatter(test_data['poverty_index'], predictions, marker = 'o', color = 'red')

# Inspección cualitativa
residuals = results.resid
sm.qqplot(residuals, line = 'r', ax = axes[1])
axes[1].set(title='qqplot') # los residuales no son normales (los puntos azules no siguen la linea roja)

# Tests de heterocedasticidad
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white
labels = ['LM-Statistic','LM-Test p-value', 'F-Statistic', 'F-Test p-value']
white_test = het_white(residuals,  model.exog)
bp_test = het_breuschpagan(residuals, model.exog)
print(dict(zip(labels[2:4], bp_test[2:4]))) # p<0.05 hay heterocedasticidad
print(dict(zip(labels[2:4], white_test[2:4]))) # p<0.05 hay heterocedasticidad

#Incluyamos errores estandar robustos
model = smf.ols(formula='health_expenditures ~ poverty_index*educ_hh', data = HISP) 
results = model.fit(cov_type='HC3')

plt.show()
print(results.summary()) 



#### Alternativa: regresión por percentiles

Una alternativa, es analizar los datos por percentiles. Por ejemplo, en la gráfica anterior se podría analizar sólo los que más gastan en salud. Veamos como se haría.

In [ ]:
# Regresión por percentiles
model = smf.quantreg(formula='health_expenditures ~ poverty_index*educ_hh', data = train_data)
def fq(q):
    fig, ax = plt.subplots(figsize=(8, 6))
    sn.scatterplot(x = 'poverty_index', y='health_expenditures', data=HISP, ax = ax)
    results = model.fit(q=q)
    predictions = results.predict(test_data.loc[:,['Intercept','poverty_index','educ_hh','poverty_index:educ_hh']])
    ax.scatter(test_data['poverty_index'], predictions, marker = 'o', color = 'red')
    plt.show()
    print(results.summary())
    

# Visualización
interact(fq, q = widgets.FloatSlider(min = 0.025, max = 0.975, step = .025, value = 0.5,
                                    description = 'Percentile'))

# Ponga el percentile en el menor valor con el slider. 
# Note como las predicciones de la regresión (linea roja) solo capturan el percentile más bajo de la variable dependiente (health_expenditure)



### Ejercicio 1 - Regresión

Hemos aprendido que las regresiones pueden tener estimativos de coeficientes sesgados. En este ejercicio vamos a correr una regresión con heterocedasticidad. Haga lo siguiente:

* Cargue la siguiente data con el siguiente comando: 
    * data = sm.datasets.engel.load_pandas().data

* Inspeccione la data usando el método .head()

* Haga una gráfica donde el eje x sea `income` y el eje y `expenditure`. Observando la gráfica, piensa que hay heterocedasticidad. ¿Por qué?

* Haga una regresión tradicional donde la variable endógena sea `expenditure` y la exógena `income`. Con el comando print muestre los resultados del ajuste.

* Haga un test de heterocedasticidad. ¿Qué le dice el p-valor?

* Ahora ajuste el modelo con errores estandar robustos. ¿Qué cambio relativo a la regresión tradicional?

* En la gráfica que hizo antes, ponga las predicciones de la regresión con una línea roja.

## Regresiones y causalidad (cercanas a...)  

¿Cómo sabemos si nuestro estimativo lineal está sesgado i.e. no refleja el proceso que generó la realidad? Con seguridad lo va a estar. Son modelos lineales para fenómenos complejos. Sin embargo, vamos a aprender técnicas que aumentan nuestra confianza de que las relaciones son más robustas que una simple correlación.

### Técnica 1: Diferencia en diferencias (DiD)

In [ ]:
# Vamos a reproducir la tabla 7.3 Gertler et al 2016, WB book
# Empecemos filtrando la data HISP Health Insurance Subsidy Program
# Intervención a evaluar: subsidio para pagar seguro de salud (columna enrolled) 
# Variable a evaluar: gasto en salud del bolsillo (columna health_expenditures) 
# Por el momento, nos interesa localidades donde se intervino
msk = HISP['treatment_locality'] == 1
HISP_temp = HISP[msk].copy()
HISP_temp.head()
#print(HISP_temp.dtypes)

Antes de hacerlo en una regresión calculemos la diferencia en diferencias a mano.

In [ ]:
# El punto en el tiempo antes y después esta en la columna round
# La diferencia en diferencias sería:
# (enrolled 0 round 0 - enrolled 0 round 1) -(enrolled 1 round 0 - enrolled 1 round 1) 
g = HISP_temp.groupby(['enrolled','round']).mean().reset_index() #¿Qué hace este comando?
tratado = g['enrolled'] == 1
no_tratado = g['enrolled'] == 0
t1 = g['round'] == 1
t0 = g['round'] == 0
outcome = 'health_expenditures'
efecto_tratados =  float(g.loc[(tratado) & (t0), outcome]) - float(g.loc[(tratado) & (t1), outcome])
efecto_no_tratados =  float(g.loc[(no_tratado) & (t0), outcome]) - float(g.loc[(no_tratado) & (t1), outcome])
DiD =  round(efecto_tratados -  efecto_no_tratados, 4)
print("El efecto de la intervención fue una reducción en gastos de bolsillo en salud de " + str(DiD) )

Ahora calculemos la DiD con una regresión. Lo que vamos a ver es que la diferencia en diferencias es el termino de interacción. En el contexto de la data HISP, el DiD nos dice cuanto subio (o bajo) el gasto en bolsillo en salud en un grupo vs. el otro en dos periodos de tiempo diferente i.e. diferencia en pendientes. Y eso precisamente es la interacción.

In [ ]:
# DiD es el efecto interacción  
# Note como escribimos la formula. Hubieramos podido usar *, pero la idea es aprender
# que dos puntos (:) indica interacción. Esto es útil cuando hay más variables y no queremos 
# todas las interacciones (* hace todas y efectos solos).
model = smf.ols(formula='health_expenditures ~ round:enrolled + round + enrolled', 
                data = HISP_temp)
results = model.fit()
print(results.summary())


In [ ]:
# ¿Qué pasa si usamos errores robustos? 
# El coeficiente de la interacción (DiD) no cambia, solo los errores estandar
model = smf.ols(formula = 'health_expenditures ~ round:enrolled + round + enrolled', 
                data = HISP_temp)
results = model.fit(cov_type='HC3')
print(results.summary())

In [ ]:
# ¿Qué pasa si introducimos controles demográficos?
# Cambia la DD
model = smf.ols(formula = 'health_expenditures ~ round:enrolled + round + enrolled +' \
                'age_hh + age_sp + educ_hh + educ_sp + indigenous +' \
                'female_hh + hhsize + dirtfloor + bathroom + land + hospital_distance', 
                data = HISP_temp)
results = model.fit(cov_type='HC3')
print(results.summary())

### Ejercicio 2 - Diferencia en Diferencias (DiD)

Calcule el efecto del terrorismo en la economía del país Basco en España. Para ello usará la técnica de DiD, manualmente y con una regresión. Los datos están en la base `basque.csv`. De interés para este ejercicio, la base tiene:
* Información en el periodo 1955 - 1997
* Información de 18 regiones de España
* El año de tratamiento es 1975. Tratamiento es aparición de terrorismo (ETA se formó en 1974).
* La región tratada es 'Basque Country (Pais Vasco)'
* Mediremos el impacto sobre la variable PIB per capita (`gdpcap`; está en miles)

Su tarea es hacer lo siguiente:

1. Cargar los datos a una variable con el nombre `basque` (basque.csv está en la carpeta DataSets).
2. Crear una nueva base de datos con el nombre `basque_reducida`. Está nueva base solo tendrá las regiones `Basque Country (Pais Vasco)` y `Cataluna`. 
    * Tip 1: los nombres de las regiones están en la columna `regionname`. 
    * Tip 2: use indexación lógica (ver comienzo de esta sección) o el método `query()` de pandas dataframes.
    * Tip 3: use el operador booleano o: `|`.
3. Haga una gráfica con dos líneas, una para cada región. Eje x: `year`, Eje y: `gdpcap`. Describa qué pasa antes y después de 1975 en el País Basco relativo a Cataluña. 
    * Tip: grafique con la libreria seaborn (https://seaborn.pydata.org/generated/seaborn.lineplot.html). Recuerde que la importamos con el alias `sn`. Use el parametro 'hue'.
4. Crear variables dummy en `basque_reducida` en dos columnas nuevas con los siguientes nombres:
    - `post`: toma el valor 0 si la columna `year` es <1975, 1 de lo contrario
    - `treat`: toma el valor 0 si la columna `regionname` es Cataluna, 1 de lo contrario

5. Use el método `groupby` para calcular manualmente la DiD. Recuerde que nos interesa el PIB per capita (gdpcap). 
    * Tip: Use groupby con las columnas post y treat del punto anterior.
6. Corra una regresión con las mismas columnas post y treat para obtener la DiD; no olvide incluir la interacción en la regresión. ¿Le dio igual que el cálculo manual?


### Técnica 2: Regresiones con discontinuidades en el diseño (RDD)

In [ ]:
# Empecemos reduciendo la data bang que cargamos al comienzo (ver razón abajo)
# fuente: khandker et al 2010 worldbank book pg 212
# dmmfd: Household has male microcredit participant: 1=Y, 0=N
# dfmfd: Household has female microcredit participant: 1=Y, 0=N
# hhland: Household amount of owned land
# exptot: Household per capita total expenditure: Tk/year
# Las ultimas dos (hhland y exptot) se analizaran en escala logaritmica (para respetar supuestos de normalidad)

# The microcredit program was probabilistic/voluntary i.e. people above the land threshold
# could or not participate (dmmfd o dfmfd). To produce a sharp 
# cutoff only those below the threshold who should be in the program 
# and those above who shouldn't are kept for the analysis.

bang_d = bang.query('(hhland<50 & (dmmfd==1 | dfmfd==1)) |' \
                    '(hhland>=50 & (dmmfd==0 | dfmfd==0))')

if 'tratamiento' not in bang_d.columns:
    bang_d.insert(0, 'tratamiento', np.repeat(0, bang_d.shape[0]))
idx = (bang_d['hhland']<50) & ((bang_d['dmmfd']==1) | (bang_d['dfmfd']==1)) 
bang_d.loc[idx,'tratamiento'] = 1

Ahora hagamos la regresión

In [ ]:
cutoff = np.log(1 + 50/100) #el threshold en escala logaritmica
# lnland-cutoff: es el efecto de land normalizado a cero en el cutoff (facilita interpretación)
# Usamos I() para hacer operaciones entre variables. Es la función identidad.
model = smf.ols(formula='lexptot ~ I(lnland-cutoff)*tratamiento', 
                data = bang_d)
results = model.fit(cov_type='HC3')
print(results.summary()) 
# tratamiento no significativo no hay ATE (average treatment effect) en el cutoff (el "salto"); 
# interaccion casi significativa en el cutoff, puede haber cambio de pendiente pero no es seguro



In [ ]:
# Veamos en gráficas que de hecho no hay saltos en consumo evidentes antes y 
# después del umbral de eligibilidad para el microcredito

fig, ax = plt.subplots(figsize=(8,5))
sn.scatterplot(x = 'lnland', y = 'lexptot', hue = 'tratamiento', 
               data = bang_d, ax = ax)
if 'cutoff' not in band_d.columns:
    bang_d.insert(0,'cutoff',np.repeat(cutoff,bang_d.shape[0]))

# Izquierda del umbral
dataT = bang_d.query('lnland<=cutoff')
model = smf.ols(formula='lexptot ~ lnland', 
                data = dataT)
results = model.fit(cov_type='HC3')
predictions = results.predict(dataT['lnland'])
if 'predictions' not in dataT.columns:
    dataT.insert(0,'predictions',predictions)
sn.lineplot(x = 'lnland', y = 'predictions', color = 'black',
           data = dataT, ax = ax)

# Derecha del umbral
dataT = bang_d.query('lnland>cutoff')
model = smf.ols(formula='lexptot ~ lnland', 
                data = dataT)
results = model.fit(cov_type='HC3')
predictions = results.predict(dataT['lnland'])
if 'predictions' not in dataT.columns:
    dataT.insert(0,'predictions',predictions)
sn.lineplot(x = 'lnland', y = 'predictions', color = 'black',
           data = dataT, ax = ax)

ax.set_xlabel('Cantidad de tierra (log)')
ax.set_ylabel('Consumo (log)')

# En el punto que cambia de color los puntos, las lineas de regresión no saltan 
# No LATE: local average treatment effect. Vamos hablar más de esto luego. Basta decir que
# el efecto causal (si hay) se daria en las proximidades del umbral (donde cambian de color los puntos)
# La razón: justo antes y después del umbral se asume que los participantes son bien parecidos
# Parece haber un cambio de pendiente pero ese no nos interesa por el momento.


In [ ]:
# Calculemos un LATE (local average treatment effect) a mano.
# Necesitamos definir el ancho del umbral (bandwidth). 
# En la figura anterior, equivale a definir el rango del eje x 
# donde queremos calcular el efecto en consumo antes y después 
# del cambio de color de los puntos

bw = cutoff*0.5 #bandwidth NOTA: esto es una heuristica, hay otras que incluyen kernels triangulares, gaussianos, otros!
if 'bw' not in bang_d.columns:
    bang_d.insert(0,'bw',np.repeat(bw,bang_d.shape[0]))
dataT = bang_d.query('lnland>cutoff & lnland<(cutoff+bw) & tratamiento == 0')
R = dataT['lnland'].mean() #derecha del umbral
dataT = bang_d.query('lnland<cutoff & lnland>(cutoff-bw) & tratamiento == 1')
L = dataT['lnland'].mean() #izquierda del umbral
print("LATE en consumo (escala log.): " + str(round(R-L,2))) #escala eje y de la figura anterior
print("LATE en consumo (escala lineal): " + str(round(np.exp(R-L),2))) #

### Ejercicio 3 - Regresión discontinua (RDD)

No se puede manejar si se consumió alcohol. Para aplicar este principio, las autoridades miden el nivel de alcohol en la sangre (NAS de ahora en adelante). Si el NAS supera un umbral, la persona es sancionada. Con una regresión discontinua, Hansen (2015) encontró evidencia causal que esta medida es efectiva para reducir la reincidencia de manejar bajo los efectos del alcohol. Con una data simulada parecida a la de Hansen, vamos a hacer parte de su análisis (haga todas las gráficas con seaborn o matplotlib, escoja solo uno).

Haga lo siguiente:

1. Cargue la data `BAC.csv` (en la carpeta DataSets) en una variable llamada `BAC` . La data tiene medidas promedios de varios conductores reincidentes para distintos NAS (la unidad de análisis no es conductor, es NAS). 
2. Imprima el nombre de las columnas con el método .dtypes
3. Chequee si las variables sociodemográficas `Genero_promedio`,`Accidente_en_escena_promedio`,`Edad_promedio_estandarizada`, `Raza_blanco_promedio`, cambian en el valor del `Umbral_NAS`. Es decir, haga para cada variable una gráfica: Eje x: `Nivel_Alcohol_Sangre_NAS`. Eje y: variable. Visualmente revise si hay saltos en el umbral. Bono: ponga todas las graficas en una sola figura con plt.subplots().
3. Cree una nueva columna llamada `Distancia_a_umbral`. Para crearla, reste las columnas `Nivel_Alcohol_Sangre_NAS` y `Umbral_NAS`.
4. Corra dos regresiones lineales y guarde los resultados en distintas variables. La primera con datos que tengan `Nivel_Alcohol_Sangre_NAS` menor o igual al `Umbral_NAS`. La segunda con datos que tengan `Nivel_Alcohol_Sangre_NAS` mayor al `Umbral_NAS`
5. Haga la siguiente gráfica:
    - Eje x: `Nivel_Alcohol_Sangre_NAS`. Eje y = `Reincidencia_promedio`
    - Añada dos lineas rojas con las predicciones de las regresiones que hizo en el punto 3.
6. Corra una regresión lineal con las siguientes características:
    - Variable dependiente: `Reincidencia_promedio`
    - Variables independientes: `Incumplio_umbral`, `Distancia_a_umbral`, y la interacción
    - Errores estandar robustos
7. Interprete los resultados. ¿Cuál parámetro nos indica la discontinuidad?
8. Introduzca controles sociodemográficos en la regresión (las demás columnas).
9. Calcule el LATE alrededor del salto. Use el procedimiento que aprendimos antes con un bandwith de 0.02. 
10. Interprete el LATE.


Referencias:

Hansen, B. (2015). Punishment and deterrence: Evidence from drunk driving. American Economic Review, 105(4), 1581-1617.

### Técnica 3 - Variables instrumentales


In [ ]:
# Empezemos con un ejemplo con data simulada de la cual sabemos la formula verdadera. 
# Fuente: Health econometrics using stata; 1st edition 2017, pp = 202-

np.random.seed(123456) #esto hace que los numeros aleatorios sean los mismos cada corrida
nobs = 10000
x = np.random.normal(size = nobs) # exogenous variable 
w = np.random.normal(size = nobs) # instrumental variable
u = np.random.normal(size = nobs) # omitted variable (unobserved)
e1 = np.random.normal(size = nobs) # outcome error
e2 = np.random.normal(size = nobs) # endogenous error
y2 = x + .2*w + u + e2 # endogenous equation. y2 es una variable de interés que afecta y1 (e.g. educación) 
y1 = y2 + x + u + e1 # outcome equation (e.g. y1 es colesterol). No la conocemos, por eso usamos regresiones.
data_dict = {'x': x, 'w': w, 'u': u, 'y2': y2, 'y1': y1}
data_sim = pd.DataFrame(data_dict)

# La regresión con u da un buen estimativo de como y2 afecta y1
# i.e. sabemos que es 1 por el outcome equation que definimos
model = smf.ols("y1 ~ y2 + x + u", data = data_sim) #Esta es la regresión que hariamos si tuvieramos TODAS las variables
results = model.fit(cov_type='HC3')
print(results.summary())


In [ ]:
# La regresión sin u (variables no observadas) da un estimativo sesgado del coeficiente de y2. 
# Razón: las variables no observadas (u) y la variable independiente y2
# se correlacionan positivamente (ver endogenous equation). 
# Es decir, y2 carga información de variables no observadas. Al no incluir u, 
# la regresión asigna a y2 otra relevancia de lo que verdad tiene. 

# Por ejemplo, si y1 es colesterol, y2 es educación, y u es salud mental,
# entonces al quitar salud mental de la ecuación, y asumamos que salud mental se
# correlaciona con niveles de educación obtenidos, el efecto de educación
# se estima mal pues no incluye la variable no observada que afecta tanto a
# y1 (colesterol) y y2 (educación).

model = smf.ols("y1 ~ y2 + x", data = data_sim)
results = model.fit(cov_type='HC3')
print(results.summary())

La pregunta natural es si u son variables no observadas ¿No estamos atascados entonces? Una solución es usar variables instrumentales (IV, por sus siglas en inglés). Note en la formula de y2 que hay una variable w. Esta variable tiene propiedades interesantes:

* Se correlaciona con y2. 
* No se correlaciona  con u (no observadas).
* Afecta a y1 via y2.

Como vimos en clase, esas características son claves para pensar a w como una variable instrumental. La idea básica es que vamos a reemplazar y2 usando w. Ese reemplazo se denomina instrumentalizar a y2 con w. Una forma de hacerlo es con 2SLS.  Veamos como implementamos esta idea para reducir el sesgo de no incluir variables no observadas en la regresión. 

In [ ]:
# Variables instrumentales via 2SLS (Two-stage least squares)
etapa1 = smf.ols('y2 ~ x + w', data = data_sim) #stage 1: 
results = etapa1.fit(cov_type='HC3')
#print(results.summary())
y2_hat = results.predict(data_sim.loc[:,['x','w']])
if 'y2_hat' not in data_sim.columns:
    data_sim.insert(0, 'y2_hat', y2_hat)
etapa2 = smf.ols('y1 ~ x + y2_hat', data = data_sim) #stage 2:
results = etapa2.fit(cov_type='HC3')
print(results.summary())

# Note como el coeficiente de y2_hat y x son más cercanos a lo definido en outcome equation.
# A pesar de no observar variables criticas (u), y2_hat ya incluia esa información (i.e. u va al ruido de la etapa1).
# El poder de este procedimiento es que soluciona el problema de variables omitidas.
# El reto es encontrar buenos instrumentos w (ver anterior markdown para qué es un buen instrumento)

In [ ]:
# Es recomendable usar funciones para hacer de forma directa la 2SLS. 
# Estas funciones están optimizadas, evita errores, tienen métodos adicionales,  
# y vuelve fácil leer su código para usos futuros. 
if 'Intercept' not in data_sim.columns:
    data_sim.insert(0, 'Intercept', np.repeat(1 ,data_sim.shape[0]))
resultsIV = IV2SLS(dependent = data_sim['y1'],
                   exog = data_sim[['Intercept','x']],
                   endog = data_sim['y2'],
                   instruments = data_sim['w']).fit()
print(resultsIV.summary)

In [ ]:
# ¿Como saber si necesitamos instrumentos y son buenos? Primer filtro: sentido común. Pero también hay tests.
# Algunos tests importantes de exogeneidad, bondad de instrumentos, y especificación.

# Empecemos añadiendo un instrumento nuevo
if 'w2' not in data_sim.columns: #otro instrumento
    data_sim.insert(0, 'w2', data_sim['w']**2)
instruments = ['w', 'w2']
resultsIV = IV2SLS(dependent = data_sim['y1'],
                   exog = data_sim[['Intercept','x']],
                   endog = data_sim['y2'],
                   instruments = data_sim[instruments]).fit()

print(resultsIV.summary)
print('\n')
print(resultsIV.wu_hausman()) #p<0.05 hay endogeneidad en el OLS i.e. incluir variable instrumental ayuda
print('\n')
print(resultsIV.wooldridge_regression) #p<0.05 hay endogeneidad en el OLS i.e. incluir variable instrumental ayuda
print('\n')
print(resultsIV.sargan) #p<0.05 significa que uno o varios de los instrumentos no se necesitan (sobreidentificacion)
print('\n')
print(data_sim[['y2'] + instruments].corr()) #correlacion entre endogenas e instrumentos tiene que ser alta para ser buen instrumento 

# Los tests nos dicen que necesitamos instrumentos.
# Sin embargo, el instrumento w2 no es importante, la correlación es baja con la endogena y2.



### Ejercicio 4 - Variables instrumentales

La data que vamos a usar en este ejercicio tiene gastos en salud del bolsillo y varias características individuales. Haga una regresión con variables instrumentales.

1. Cargue los datos `mus06data.dta` en una variable llamada mi_data. El archivo está en la carpeta DataSets. Recuerde, para cargar datos con la terminación .dta debe usar pd.read_stata().
2. Obtenga la descripción de las columnas. Para ello cree otra variable llamada mi_data_descripcion. Use de nuevo pd.read_stata() pero ahora ponga la opción iterator = True. Luego escriba en una nueva linea el siguiente comando: mi_data_descripcion.variable_labels(). Lea y entienda la descripción de las siguientes variables: 
    - `ldrugexp ` (variable dependiente)
    - `totchr age female blhisp linc` (variables exógenas)
    - `hi_empunion` (variable endógena a instrumentalizar)
    - `ssiratio, lowincome, firmsz, multlc` (instrumentos)
3. Antes de calcular cualquier estadistica, sustente por qué `ssiratio` podría ser un buen instrumento para `hi_empunion`. SSI es Supplemental Security Income. Es un programa que paga beneficios a adultos y niños que tienen ingresos y recursos limitados (https://www.ssa.gov/ssi/).
4. Calcule la correlación entre los instrumentos y la variable endógena. ¿Cuál parece ser el mejor y peor instrumento?
5. Haga una regresión lineal simple con las variables descritas en el punto 2, SIN los instrumentos. Imprima la tabla de resultados. Observe el coeficiente de `hi_empunion`.
6. Ahora haga una regresión lineal con la variable instrumental `ssiratio`. Use la función IV2SLS. En caso que no la haya importado, este es el comando: from linearmodels.iv import IV2SLS.
7. Comente la diferencia entre los coeficientes para `hi_empunion` de la regresión hecha para el punto 5 y 6. ¿Por qué difieren?
8. Haga una regresion con todas las variables instrumentales `ssiratio, lowincome, firmsz, multlc`. Para esta regresión corra los tests de wu_hausman(), wooldridge_regression, y de sargan. ¿Qué significan?
